[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/arpm92/IAH_IA-Base/blob/main/gen-imagenes.ipynb)


# **Difusión Estable** 🎨
*...using `🧨diffusers`*

Stable Diffusion es un modelo de difusión latente texto-imagen creado por los investigadores e ingenieros de [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai/) y [LAION](https://laion.ai/). Está entrenado con imágenes de 512x512 de un subconjunto de la base de datos [LAION-5B](https://laion.ai/blog/laion-5b/). Este modelo utiliza un codificador de texto congelado CLIP ViT-L/14 para condicionar el modelo a las indicaciones de texto. Con sus 860M de UNet y 123M de codificador de texto, el modelo es relativamente ligero y puede ejecutarse en muchas GPU de consumo.
Consulta la [ficha del modelo](https://huggingface.co/CompVis/stable-diffusion) para obtener más información.

Este cuaderno de Colab muestra cómo usar Stable Diffusion con la 🤗 Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers).

¡Vamos a empezar!

## 1. Cómo utilizar `StableDiffusionPipeline

Antes de sumergirnos en los aspectos teóricos de cómo funciona Stable Diffusion,
vamos a probarlo un poco 🤗.

En esta sección, mostramos cómo puede ejecutar la inferencia de texto a imagen ¡en sólo unas pocas líneas de código!

### Configuración

En primer lugar, por favor, asegúrese de que está utilizando un runtime GPU para ejecutar este cuaderno, por lo que la inferencia es mucho más rápida. Si el siguiente comando falla, utilice el menú `Runtime` de arriba y seleccione `Change runtime type`.

In [ ]:
!nvidia-smi

A continuación, debes instalar `diffusers` así como `scipy`, `ftfy` y `transformers`. Se utiliza `accelerate` para conseguir una carga mucho más rápida.

In [ ]:
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

### Tubería de difusión estable

`StableDiffusionPipeline` es un proceso de inferencia de extremo a extremo que puede utilizar para generar imágenes a partir de texto con sólo unas pocas líneas de código.

Primero, cargamos los pesos pre-entrenados de todos los componentes del modelo. En este cuaderno utilizamos la versión 1.4 de Stable Diffusion ([CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4)), pero hay otras variantes que puede probar:
* [runwayml/stable-diffusion-v1-5](https://huggingface.co/runwayml/stable-diffusion-v1-5)
* [stabilityai/stable-diffusion-2-1-base](https://huggingface.co/stabilityai/stable-diffusion-2-1-base)
* [stabilityai/stable-diffusion-2-1](https://huggingface.co/stabilityai/stable-diffusion-2-1). Esta versión puede producir imágenes con una resolución de 768x768, mientras que las demás funcionan a 512x512.

Además del id del modelo [CompVis/stable-diffusion-v1-4](https://huggingface.co/CompVis/stable-diffusion-v1-4), también pasamos una `revision` y un `torch_dtype` específicos al método `from_pretrained`.

Queremos asegurarnos de que cada Google Colab libre puede ejecutar Stable Diffusion, por lo que estamos cargando los pesos desde la rama de media precisión [`fp16`](https://huggingface.co/CompVis/stable-diffusion-v1-4/tree/fp16) y también le decimos a `diffusers` que espere los pesos en precisión float16 pasando `torch_dtype=torch.float16`.

Si desea asegurar la mayor precisión posible, por favor asegúrese de eliminar `torch_dtype=torch.float16` a costa de un mayor uso de memoria.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)

In [ ]:
pipe = pipe.to("cuda")

In [ ]:
Y ya estamos listos para generar imágenes:

In [ ]:
prompt = "a photograph of an astronaut riding a horse"
image = pipe(prompt).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

# Now to display an image you can either save it such as:
image.save(f"astronaut_rides_horse.png")

# or if you're in a google colab you can directly display it with
image

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, generator=generator).images[0]

image

Puede cambiar el número de pasos de inferencia utilizando el argumento `num_pasos_de_inferencia`. En general, los resultados son mejores cuantos más pasos se utilicen. Stable Diffusion, siendo uno de los modelos más recientes, funciona muy bien con un número relativamente pequeño de pasos, por lo que recomendamos utilizar el valor por defecto de `50`. Si desea resultados más rápidos puede utilizar un número menor.

La siguiente celda utiliza la misma semilla que antes, pero con menos pasos. Observe cómo algunos detalles, como la cabeza del caballo o el casco, son menos realistas y están menos definidos que en la imagen anterior:

In [ ]:
import torch

generator = torch.Generator("cuda").manual_seed(1024)

image = pipe(prompt, num_inference_steps=15, generator=generator).images[0]

image

Para generar múltiples imágenes para el mismo prompt, simplemente usamos una lista con el mismo prompt repetido varias veces. Enviaremos la lista a la tubería en lugar de la cadena que usamos antes.

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

In [ ]:
num_images = 3
prompt = ["a photograph of an astronaut riding a horse"] * num_images

images = pipe(prompt).images

grid = image_grid(images, rows=1, cols=3)
grid

A continuación se explica cómo generar una cuadrícula de n × m imágenes.

In [ ]:
num_cols = 3
num_rows = 4

prompt = ["a photograph of an astronaut riding a horse"] * num_cols

all_images = []
for i in range(num_rows):
  images = pipe(prompt).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid